In [11]:
import mysql.connector
from mysql.connector import errorcode
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Date , Float, Table, Text, SmallInteger, UniqueConstraint,func
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
import pandas as pd
import ast
import logging
from datetime import datetime
from sqlalchemy.engine import URL
from urllib.parse import quote_plus
import unicodedata
from rapidfuzz import process, fuzz


In [41]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="server@#@898"
)
cursor = conn.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS Basketball_Reference")
print("Database created (or already exists).")

conn.close()

Database created (or already exists).


In [43]:
DB_NAME = "Basketball_Reference"

config = {
  'host': 'localhost',
  'user': 'root',
  'password': 'server@#@898', 
  'database' : DB_NAME
  
}


print("Connection parameters defined.")

Connection parameters defined.


In [ ]:

df_players = pd.read_csv("players_info.csv")

df_players['PlayerID'] = range(1, len(df_players) + 1)
df_players_clean = df_players.drop(columns=['Positions', 'Current_team'])

positions_set = set()
for pos_list in df_players['Positions']:
    if pd.notna(pos_list) and pos_list.strip() not in ["", "[]"]:
        for pos in eval(pos_list):
            if pos.strip() != "":
                positions_set.add(pos)

df_positions = pd.DataFrame({
    'PositionID': range(1, len(positions_set) + 1),
    'PositionName': list(positions_set)
})

player_positions = []
for _, row in df_players.iterrows():
    player_id = row['PlayerID']
    pos_list = row['Positions']
    if pd.notna(pos_list) and pos_list.strip() not in ["", "[]"]:
        for pos in eval(pos_list):
            if pos.strip() != "":
                pos_id = df_positions[df_positions['PositionName'] == pos]['PositionID'].values[0]
                player_positions.append({'PlayerID': player_id, 'PositionID': pos_id})

df_player_positions = pd.DataFrame(player_positions)


teams_unique = df_players['Current_team'].dropna().unique()
df_teams = pd.DataFrame({
    'TeamID': range(1, len(teams_unique) + 1),
    'TeamName': teams_unique
})


player_teams = []
for _, row in df_players.iterrows():
    player_id = row['PlayerID']
    team_name = row['Current_team']
    if pd.notna(team_name):
        team_id = df_teams[df_teams['TeamName'] == team_name]['TeamID'].values[0]
        player_teams.append({'PlayerID': player_id, 'TeamID': team_id})

df_player_teams = pd.DataFrame(player_teams)

df_players_clean.to_excel("Players.xlsx", index=False)                
df_positions.to_excel("Positions.xlsx", index=False)                 
df_player_positions.to_excel("Player_Positions.xlsx", index=False)    
df_teams.to_excel("Teams.xlsx", index=False)                          
df_player_teams.to_excel("Player_Teams.xlsx", index=False)            


In [26]:
df1 = pd.read_excel("Teams.xlsx")
df2 = pd.read_csv("team_infos.csv")
df_merged = pd.merge(df1, df2, left_on="TeamName", right_on="Team", how="left")
df_merged.drop(columns=["Team"], inplace=True)
df_merged.to_excel("Teams_Final.xlsx", index=False)

In [27]:

def normalize_name(name):
    if pd.isna(name):
        return ""
    return unicodedata.normalize('NFKD', str(name)).encode('ascii', 'ignore').decode('ascii').strip().lower()


df_team  = pd.read_excel("Teams_Final.xlsx")       
df_players = pd.read_excel("Players.xlsx")
df_champions = pd.read_csv("champions.csv")

df_players['Name_norm'] = df_players['Name'].apply(normalize_name)
players_list_norm = df_players['Name_norm'].tolist()

df_champions.columns = df_champions.columns.str.strip()  
seasons_unique = df_champions['season'].drop_duplicates().reset_index(drop=True)
df_seasons = pd.DataFrame({
    'SeasonID': range(1, len(seasons_unique)+1),
    'Season': seasons_unique
})
df_champions = df_champions.merge(df_seasons, left_on='season', right_on='Season', how='left')


champions_unique = df_champions['Champion'].drop_duplicates().reset_index(drop=True)
df_champion_name = pd.DataFrame({'ChampionName': champions_unique})


df_champion_id = df_champion_name.merge(
    df_team[['TeamName', 'TeamID']],
    left_on='ChampionName',
    right_on='TeamName',
    how='left'
)


df_champion_id['ChampionID'] = range(1, len(df_champion_id)+1)
df_champion_id = df_champion_id[['ChampionID', 'ChampionName', 'TeamID']]

df_champions = df_champions.merge(df_champion_id, left_on='Champion', right_on='ChampionName', how='left')

rows = []
for idx, row in df_champions.iterrows():
    season_id = row['SeasonID']
    champion_id = row['ChampionID']
    players_str = row['players']
    
    try:
        players_list = ast.literal_eval(players_str)
    except:
        players_list = []
    
    for player_name in players_list:
        player_name_norm = normalize_name(player_name)
        match, score, match_idx = process.extractOne(player_name_norm, players_list_norm, scorer=fuzz.ratio)
        
        if score >= 80:  
            player_id = df_players.iloc[match_idx]['PlayerID']
            rows.append({
                'SeasonID': season_id,
                'ChampionID': champion_id,
                'PlayerID': player_id
            })
        else:
            print(f"Player not found (low similarity): {player_name}")

df_final_players = pd.DataFrame(rows)


df_seasons.to_excel("seasons.xlsx", index=False)
df_champion_id.to_excel("Champions_ID.xlsx", index=False)
df_final_players.to_excel("Champions_Players.xlsx", index=False)


In [28]:


def normalize_name(name):
    if pd.isna(name):
        return ""
    return unicodedata.normalize('NFKD', str(name)).encode('ascii', 'ignore').decode('ascii').strip().lower()


df_players = pd.read_excel("Players.xlsx") 
df_seasons = pd.read_excel("Seasons.xlsx")  


df_players['Name_norm'] = df_players['Name'].apply(normalize_name)
players_list_norm = df_players['Name_norm'].tolist()


df_ranks = pd.read_csv("players_ranked.csv") 
df_ranks['name_norm'] = df_ranks['name'].apply(normalize_name)


df_ranks = df_ranks.merge(df_seasons, left_on='season', right_on='Season', how='left')


player_ids = []
for name in df_ranks['name_norm']:
    match, score, idx = process.extractOne(name, players_list_norm, scorer=fuzz.ratio)
    if score >= 80:
        player_ids.append(df_players.iloc[idx]['PlayerID'])
    else:
        player_ids.append(None)
        print(f"Player not found (low similarity): {name}")

df_ranks['PlayerID'] = player_ids


df_final_ranks = df_ranks[['rank', 'PlayerID', 'SeasonID']]


df_final_ranks.to_excel("Player_Ranks.xlsx", index=False)



In [29]:
def normalize_name(name):
    if pd.isna(name):
        return ""
    return unicodedata.normalize('NFKD', str(name)).encode('ascii', 'ignore').decode('ascii').strip().lower()


df_players = pd.read_excel("Players.xlsx") 
df_seasons = pd.read_excel("Seasons.xlsx")   

df_players['Name_norm'] = df_players['Name'].apply(normalize_name)
players_list_norm = df_players['Name_norm'].tolist()

df_ranks = pd.read_csv("Michael_Jordan_Trophy.csv")  
df_ranks['name_norm'] = df_ranks['name'].apply(normalize_name)

df_ranks = df_ranks.merge(df_seasons, left_on='season', right_on='Season', how='left')


player_ids = []
for name in df_ranks['name_norm']:
    match, score, idx = process.extractOne(name, players_list_norm, scorer=fuzz.ratio)
    if score >= 65:
        player_ids.append(df_players.iloc[idx]['PlayerID'])
    else:
        player_ids.append(None)
        print(f"Player not found (low similarity): {name}")

df_ranks['PlayerID'] = player_ids

df_final_ranks = df_ranks[['rank', 'PlayerID', 'SeasonID']]

df_final_ranks.to_excel("Michael_Jordan_Trophy.xlsx", index=False)

In [44]:
Base = declarative_base()

class Player(Base):
    __tablename__ = "players"

    PlayerID = Column(Integer, primary_key=True, autoincrement=True)
    Name = Column(String(100), nullable=False)
    Birth_date = Column(Date, nullable=False)
    Age = Column(Integer)
    Status = Column(String(20))
    Height = Column(Integer)
    Weight = Column(Integer)
    Nationality = Column(String(50))
    Shoots = Column(String(10))
    Experience = Column(Integer)
    PTS = Column(Float)


    player_positions = relationship("PlayerPositions", back_populates="player", cascade="all, delete-orphan")
    player_teams = relationship("PlayerTeam", back_populates="player", cascade="all, delete-orphan")
    champion_players = relationship("ChampionPlayers", back_populates="player", cascade="all, delete-orphan")
    player_ranks = relationship("PlayerRanks", back_populates="player", cascade="all, delete-orphan")
    michael_jordan_trophies = relationship("MichaelJordanTrophy", back_populates="player", cascade="all, delete-orphan")

class Position(Base):
    __tablename__ = "positions"

    PositionID = Column(Integer, primary_key=True, autoincrement=True)
    PositionName = Column(String(50), unique=True)


    player_positions = relationship("PlayerPositions", back_populates="position", cascade="all, delete-orphan")

class PlayerPositions(Base):
    __tablename__ = "player_positions"

    PlayerID = Column(Integer, ForeignKey("players.PlayerID"), primary_key=True)
    PositionID = Column(Integer, ForeignKey("positions.PositionID"), primary_key=True)

    player = relationship("Player", back_populates="player_positions")
    position = relationship("Position", back_populates="player_positions")


class Team(Base):
    __tablename__ = "teams"

    TeamID = Column(Integer, primary_key=True, autoincrement=True)
    TeamName = Column(String(100), unique=True)
    Location = Column(String(100))
    Founded = Column(String(20))  
    Championships = Column(Integer)
    Playoff_Appearances = Column(Integer)


    player_teams = relationship("PlayerTeam", back_populates="team", cascade="all, delete-orphan")

class PlayerTeam(Base):
    __tablename__ = "player_teams"

    PlayerID = Column(Integer, ForeignKey("players.PlayerID"), primary_key=True)
    TeamID = Column(Integer, ForeignKey("teams.TeamID"), primary_key=True)

    player = relationship("Player", back_populates="player_teams")
    team = relationship("Team", back_populates="player_teams")


class Season(Base):
    __tablename__ = "seasons"

    SeasonID = Column(Integer, primary_key=True, autoincrement=True)
    Season = Column(String(20), unique=True)


    champion_players = relationship("ChampionPlayers", back_populates="season", cascade="all, delete-orphan")
    player_ranks = relationship("PlayerRanks", back_populates="season", cascade="all, delete-orphan")
    michael_jordan_trophies = relationship("MichaelJordanTrophy", back_populates="season", cascade="all, delete-orphan")


class Champion(Base):
    __tablename__ = "champions"

    ChampionID = Column(Integer, primary_key=True, autoincrement=True)
    ChampionName = Column(String(100), unique=True)
    TeamID = Column(Integer, ForeignKey("teams.TeamID"))  

    champion_players = relationship("ChampionPlayers", back_populates="champion", cascade="all, delete-orphan")
    team = relationship("Team")


class ChampionPlayers(Base):
    __tablename__ = "champion_players"

    SeasonID = Column(Integer, ForeignKey("seasons.SeasonID"), primary_key=True)
    ChampionID = Column(Integer, ForeignKey("champions.ChampionID"), primary_key=True)
    PlayerID = Column(Integer, ForeignKey("players.PlayerID"), primary_key=True)

    champion = relationship("Champion", back_populates="champion_players")
    player = relationship("Player", back_populates="champion_players")
    season = relationship("Season", back_populates="champion_players")


class PlayerRanks(Base):
    __tablename__ = "player_ranks"

    RankID = Column(Integer, primary_key=True, autoincrement=True)
    Rank = Column(Integer, nullable=False)
    PlayerID = Column(Integer, ForeignKey("players.PlayerID"), nullable=False)
    SeasonID = Column(Integer, ForeignKey("seasons.SeasonID"), nullable=False)

    player = relationship("Player", back_populates="player_ranks")
    season = relationship("Season", back_populates="player_ranks")


class MichaelJordanTrophy(Base):
    __tablename__ = "michael_jordan_trophy"

    TrophyID = Column(Integer, primary_key=True, autoincrement=True)
    Rank = Column(Integer, nullable=False)
    PlayerID = Column(Integer, ForeignKey("players.PlayerID"), nullable=False)
    SeasonID = Column(Integer, ForeignKey("seasons.SeasonID"), nullable=False)

    player = relationship("Player", back_populates="michael_jordan_trophies")
    season = relationship("Season", back_populates="michael_jordan_trophies")


In [45]:
password = quote_plus("server@#@898")

engine = create_engine(
    f"mysql+mysqlconnector://{config['user']}:{password}@{config['host']}/{config['database']}"
)

Base.metadata.create_all(engine)

In [46]:
df_players = pd.read_excel("Players.xlsx")         
df_positions = pd.read_excel("positions.xlsx")       
df_teams = pd.read_excel("Teams_Final.xlsx")               
df_player_positions = pd.read_excel("player_positions.xlsx")  
df_player_teams = pd.read_excel("player_teams.xlsx")          
df_seasons = pd.read_excel("Seasons.xlsx")  
df_champions = pd.read_excel("Champions_ID.xlsx")  
df_champion_players = pd.read_excel("Champions_Players.xlsx")  
df_player_ranks = pd.read_excel("Player_Ranks.xlsx")  
df_mjt = pd.read_excel("Michael_Jordan_Trophy.xlsx")  



df_players['Birth_date'] = pd.to_datetime(df_players['Birth_date'])
df_players['Birth_date'] = df_players['Birth_date'].dt.strftime('%Y-%m-%d')

df_players.drop(columns=["Death_date"], inplace=True)
if 'Unnamed: 0' in df_teams.columns:
    df_teams.drop(columns=['Unnamed: 0'], inplace=True)

In [47]:
df_players.to_sql('players', con=engine, if_exists='append', index=False)
df_positions.to_sql('positions', con=engine, if_exists='append', index=False)
df_teams.to_sql('teams', con=engine, if_exists='append', index=False)
df_player_positions.to_sql('player_positions', con=engine, if_exists='append', index=False)
df_player_teams.to_sql('player_teams', con=engine, if_exists='append', index=False)
df_seasons.to_sql('seasons', con=engine, if_exists='append', index=False)
df_champions.to_sql('champions', con=engine, if_exists='append', index=False)
df_champion_players.to_sql('champion_players', con=engine, if_exists='append', index=False)
df_player_ranks.to_sql('player_ranks', con=engine, if_exists='append', index=False)
df_mjt.to_sql('michael_jordan_trophy', con=engine, if_exists='append', index=False)

61